## Vers un système de détection automatique de la langue

Nous allons construire un systeme simple permettant de detecter automatiquement la langue d'un texte.

Pour cela vous allez vous baser sur la méthode des N-grammes, qui a déjà fait ces preuves pour la détection de la langue.

### Préparation du corpus

Nous allons tout d'abord choisir les langues avec lesquelles nous allons travailler.

1. Choisissez trois langues parmi les langues suivantes :
*anglais, dannois, espagnol, esperanto, français, italien, latin, portugais, roumain, suedois*.

2. Téléchargez quelques livres de chaque langue choisie en format "Texte brut UTF-8" à partir du site https://www.gutenberg.org/catalog/

### Calcul des unigrammes

Nous allons écrire un programme qui calcule les fréquences des mots dans les livres. Pour chaque langue, le programme produira un fichier texte qui contient les mots rencontrés dans les livres de cette langue. Ces mots seront ordonnées par ordre de leur fréquence, du plus fréquent au moins fréquent, et chaque mot se trouvera sur une nouvelle ligne.

Comme résultat final, nous aurons des fichiers texte (par ex. fr-freq-1n.txt, en-freq-1n.txt, es-freq-1n.txt), chaque fichier contenant une liste de mots en commencant par les mots les plus fréquents dans les livres analysés.

#### Etapes :

Pour chaque langue :
1. extraire les textes à partir des fichiers 
- enlever les signes de ponctuation
- découper en mots (par ex. avec texte.split())
- créer un dictionnaire contenant les mots (unigrammes) comme clés et leurs fréquences comme valeurs
- ordonner la liste des mots par rapport aux fréquences
- enregistrer le résultat dans un fichier texte

Chaque étape pourra être faite dans une fonction séparée pour mieux organiser le code et pouvoir les réutiliser par la suite.

In [21]:
import re

def extraire_texte(fichiers):
    """Fonction qui extrait les textes a partir d'une liste de fichiers"""
    texte = ""
    for nom_fichier in fichiers:
        f = open(nom_fichier, "r", encoding="utf-8")
        texte += f.read()
        f.close()
    return texte

def ponctuation(texte):
    """Fonction qui enleve la ponctuation du texte et le convertit
    en minuscules."""
    texte = re.sub(r"[\.\-!\?,;:'\"]", "", texte)
    texte = texte.lower()
    return texte

def decouper_en_mots(texte):
    """Fonction qui decoupe en mots"""
    return texte.split()

def unigrammes(liste_mots):
    """Fonction qui produit un dictionnaire d'unigrammes"""
    d = {}
    for mot in liste_mots :
        if mot in d:
            d[mot] += 1
        else :
            d[mot] = 1
    return d

def trier_dictionnaire(d):
    """Fonction qui trie un dictionnaire par rapport aux valeurs"""
    d = {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}
    return d

def enregistrer(d, nom_fichier, limit):
    """Fonction qui enregistre le debut d'un dictionnaire dans un fichier texte"""
    f = open(nom_fichier, "w")
    counter = 0
    for k in d:
        counter += 1
        f.write(k + "\n")
        if (counter > limit) :
            break
    f.close()
    
    
texte_fr = extraire_texte( ["livres/fr-5423-0.txt", "livres/fr-pg19657.txt"] )
texte_es = extraire_texte( ["livres/es-2000-0.txt", "livres/es-57982-0.txt"] )
texte_sw = extraire_texte( ["livres/sw-56967-0.txt", "livres/sw-pg14073.txt"] )

textes = [ texte_fr, texte_es, texte_sw ]
langues = ["fr", "es", "sw"]

# traitement des unigrammes
for i in range(3):
    t = textes[i]
    langue = langues[i]
    t = ponctuation(t)
    liste_mots = decouper_en_mots(t)
    
    dict_unigr = unigrammes(liste_mots)
    dict_unigr = trier_dictionnaire(dict_unigr)
    
    enregistrer(dict_unigr, "livres/"+langue+"-1grams.txt", 500)
    

### Calcul des bi-grammes

Cette fois-ci nous nous intéressons à des suites de deux mots dans les livres. Le programme produira un fichier texte, pour chaque langue, qui contient les suites de deux mots (ou bigrammes) rencontrées dans les livres de cette langue. Ces bigrammes seront ordonnées par ordre de leur fréquence, de la plus fréquente à la moins fréquente, et chaque bigramme se trouvera sur une nouvelle ligne.

Comme résultat final, nous aurons des fichiers texte (par ex. fr-freq-2n.txt, en-freq-2n.txt, es-freq-2n.txt), chaque fichier contenant une liste de bigrammes en commencant par celles les plus fréquentes dans les livres analysés.

In [22]:
def bigrammes(liste_mots):
    """Fonction qui retourne la liste des bi-grammes et leurs frequences."""
    d = {}
    for i in range(len(liste_mots)-1):
        # construction de la bi-gramme
        bigramme = liste_mots[i] + " " + liste_mots[i+1]
        # ajout dans le dictionnaire
        if bigramme in d:
            d[bigramme] += 1
        else :
            d[bigramme] = 1
    return d

#### Etapes :

A partir de la liste des mots (étape 3 plus haut) :
7. créer la liste des bi-grammes
- créer un dictionnaire contenant les bi-grammes comme clés et leurs fréquences comme valeurs
- ordonner la liste par rapport aux fréquences
- enregistrer le résultat dans un fichier texte

Chaque étape pourra être faite dans une fonction séparée pour mieux organiser le code et pouvoir les réutiliser par la suite.

In [23]:
# traitement des bigrammes
for i in range(3):
    t = textes[i]
    langue = langues[i]
    t = ponctuation(t)
    liste_mots = decouper_en_mots(t)
    
    dict_bigr = bigrammes(liste_mots)
    dict_bigr = trier_dictionnaire(dict_bigr)
    
    enregistrer(dict_bigr, "livres/"+langue+"-2grams.txt", 500)

### Calcul de la similarité entre listes de fréquences

Ecrivez une fonction qui permet de calculer la similarité entre 2 listes ordonnées. Les listes pourront contenir des mots ou des bigrammes. On attribue un poids à chaque élement des 2 listes : ce poids est égal au nombre d'élements de la liste la plus courte et diminue d'un pour chaque position (voir les exemples ci-dessous). La fonction calculera la valeur S pour les deux listes L1 et L2 en prennant la somme des produits des poids des éléments communs. 

Voici un exemple :

L1 = [a, b, c, d, e]		avec poids [5, 4, 3, 2, 1]

L2 = [a, b, f, c, g, d]		avec poids [5, 4, 3, 2, 1, 0]


S(L1, L2) = 5 * 5 +4 * 4 + 2 * 3 + 0 * 2 = 25 + 16 + 6 + 0 = 47

(Pour chaque élément commun, on ajoute le produit de son poids dans L2 et dans L1)

Autre exemple :

L1 = [a, b, c, d, e]			avec poids [5, 4, 3, 2, 1]

L3 = [b, f, g, h, d, a, b, c]		avec poids [5, 4, 3, 2, 1, 0, 0, 0]

S(L1, L3) = 4 * 5 + 1 * 2 + 0 * 5 + 0 * 4 + 0 * 3 = 20 + 2 + 0 + 0 + 0 = 22

In [36]:
def simListe(l1, l2):
    sim = 0
    
    # on considère la longeur de la liste la plus courte
    l = len(l1)
    if (l>len(l2)) :
        l = len(l2)
    
    # une boucle pour tous les elements de la liste la plus courte
    for i in range(l):
        # element de l1
        el = l1[i]
        # poids de cet element :
        p1 = l-i
        
        if (el in l2):
            # position de l'element dans l2
            pos = l2.index(el)
            # poids de cet element dans l2
            p2 = l - pos
            
            if (p2>0) :
                sim += p1*p2
    
    return sim       
        

In [20]:
# test de la fonction simListe()
L1 = ["a", "b", "c", "d", "e"]
L2 = ["a", "b", "f", "c", "g", "d"]
L3 = ["b", "f", "g", "h", "d", "a", "b", "c"]

print (simListe(L1, L2))
print (simListe(L1, L3))

47
22


### Programme final

Ecrivez le programme final qui permet d'analyser un nouveau texte (dans une langue inconnue) qui se trouve dans un fichier entree.txt. Pour tester le programme, ce texte peut être assez court (une phrase ou un paragraphe). 

Comparez le résultat de votre programme avec le démontrateur qui se trouve ici :  https://detectlanguage.com/ (demo en bas a droite de la page).


In [44]:
# lecture du fichier entree.txt et traitement 
texte_entree = extraire_texte( ["entree.txt"] )
texte_entree = ponctuation(texte_entree)
liste_mots = decouper_en_mots(texte_entree)

# unigrammes du texte entree.txt
dict_unigr = unigrammes(liste_mots)
dict_unigr = trier_dictionnaire(dict_unigr)

liste_unigr = list(dict_unigr.keys())

# bigrammes du texte entree.txt
dict_bigr = bigrammes(liste_mots)
dict_bigr = trier_dictionnaire(dict_bigr)

liste_bigr = list(dict_bigr.keys())

# comparaison de la liste obtenue avec les differentes listes par langue :
sim_unigr = []
sim_bigr = []

for i in range(3):
    langue = langues[i]

    # extraction des unigrammes et bigrammes pour cette langue à partir des fichiers
    langue_unigr = []
    f = open("livres/"+langue+"-1grams.txt")
    for ligne in f :
        langue_unigr.append(ligne.strip())
    f.close()
    
    langue_bigr = []
    f = open("livres/"+langue+"-2grams.txt")
    for ligne in f :
        langue_bigr.append(ligne.strip())
    f.close()
    
    # calcul des similarites :
    sim1 = simListe(liste_unigr, langue_unigr)
    sim2 = simListe(liste_bigr, langue_bigr)
    
    sim_unigr.append(sim1)
    sim_bigr.append(sim2)

# Affichage du resultat : la langue la plus probable est celle où la similarité est la plus grande
print("Similarités des unigrammes :")
for i in range(3):
    print(langues[i] + " : " + str(sim_unigr[i]))
print("Langue la plus probable :")
max_val = max(sim_unigr)
pos = sim_unigr.index(max_val)
langue = langues[pos]
print(langue)

print("Similarités des bigrammes :")
for i in range(3):
    print(langues[i] + " : " + str(sim_bigr[i]))
print("Langue la plus probable :")
max_val = max(sim_bigr)
pos = sim_bigr.index(max_val)
langue = langues[pos]
print(langue)
    
    

Similarités des unigrammes :
fr : 59921
es : 65684
sw : 33722
Langue la plus probable :
es
Similarités des bigrammes :
fr : 0
es : 0
sw : 0
Langue la plus probable :
fr
